In [1]:
import pandas as pd
import os
from Test_Functions import ReadCSV

## Read all new general feature and set them in csv data set

In [2]:
source = open('features.results.out')
lines = [x.strip() for x in source.readlines()]
lines = ['{},{}{}'.format(lines[i],lines[i+1],'\n') for i in range(0,len(lines),2)]
lines = [i.replace('[','"[') for i in lines]
lines = [i.replace(']',']"') for i in lines]
target = open('semifinal.csv','w')
lines = ['experiment,reply_size,layers,tau,optimizer,advantage,max_steps,activation,batch_size,totalsteps,exploration,vanish,gamma,hidden_size,train_repeat,batch_norm,seed,rwrdschem,svision,details\n']+lines
target.writelines(lines)
target.close()

## Add Experiment details.
* Train :
    * count
    * Total Reward
    * Total step
    * Total time
    
* Test:
    * count
    * Total Reward
    * Total step
    * Total time

In [3]:
x = pd.read_csv('semifinal.csv')

In [4]:
#Train information
# Episodes count
x['tr_count']=0
# Total Reward (sum over all epsidoes)
x['tr_TR']=0
# Total Time (sum over all episodes)
x['tr_TT']=0
# Total steps (sum over all episodes)
x['tr_TS']=0
#Test Information
# Episodes count
x['ts_count']=0
# Total Reward (sum over all epsidoes)
x['ts_TR']=0
# Total Time (sum over all episodes)
x['ts_TT']=0
# Total steps (sum over all episodes)
x['ts_TS']=0
# percentage of training episodes with positive reward
x['tr_s']=0
# percentage of testing episodes with positive reward
x['ts_s']=0

x['tr_AR'] = 0.0
x['tr_AS'] = 0.0
x['ts_AR'] = 0.0
x['ts_AS'] = 0.0
#training Steps median after exploration vanish.
x['tr_SMD']=0.0
#training Steps mean after exploration vanish.
x['tr_SM']=0.0
#training Reward median after exploration vanish.
x['tr_RMD']=0.0
#training Reward mean after exploration vanish.
x['tr_RM']=0.0
#testing Steps median after exploration vanish.
x['ts_SMD']=0.0
#testing Steps mean after exploration vanish.
x['ts_SM']=0.0
#testing Reward median after exploration vanish.
x['ts_RMD']=0.0
#testing Reward mean after exploration vanish.
x['ts_RM']=0.0

In [5]:
for i in x.experiment.as_matrix():
    train,test,splitpoint = ReadCSV(i,x[x.experiment==i].vanish.as_matrix())
    
    indx = x[x['experiment']==i].index[0]
    x.set_value(indx, 'tr_count',train.shape[0])
    x.set_value(indx, 'tr_TR',train[2].sum())
    x.set_value(indx, 'tr_TT',train[3].sum())
    x.set_value(indx, 'tr_TS',train[1].sum())
    
    x.set_value(indx, 'ts_count',test.shape[0])
    x.set_value(indx, 'ts_TR',test[2].sum())
    x.set_value(indx, 'ts_TT',test[3].sum())
    x.set_value(indx, 'ts_TS',test[1].sum())
    tr_tmp = train[2]
    ts_tmp = test[2]
    x.set_value(indx,['tr_s'],tr_tmp[tr_tmp>0].count()/tr_tmp.count(),)
    x.set_value(indx,['ts_s'],ts_tmp[ts_tmp>0].count()/ts_tmp.count())
    
    train = train[train[0]>=splitpoint]
    test = test[test[0]>=(splitpoint/10)]
    
    x.set_value(indx,'tr_SMD',train[1].median())
    x.set_value(indx,'tr_SM',train[1].mean())
    x.set_value(indx,'tr_RMD',train[2].median())
    x.set_value(indx,'tr_RM',train[2].mean())

    x.set_value(indx,'ts_SMD',test[1].median())
    x.set_value(indx,'ts_SM',test[1].mean())
    x.set_value(indx,'ts_RMD',test[2].median())
    x.set_value(indx,'ts_RM',test[2].mean())
    #train = train[2]
    #test = test[2]

    print(i)

427
431
426
430
429
428
432


In [6]:
x['tr_AR'] = x['tr_TR']/x['tr_count']
x['tr_AS'] = x['tr_TS']/x['tr_count']
x['ts_AR'] = x['ts_TR']/x['ts_count']
x['ts_AS'] = x['ts_TS']/x['ts_count']

In [7]:
x.to_csv('semifinal.csv',index=False)

In [9]:
x.sort_values('experiment').head(10)

,experiment,reply_size,layers,tau,optimizer,advantage,max_steps,activation,batch_size,totalsteps,...,ts_AR,ts_AS,tr_SMD,tr_SM,tr_RMD,tr_RM,ts_SMD,ts_SM,ts_RMD,ts_RM
2,426,100000,1,0.001,adam,max,1000,tanh,32,2000000,...,986.810948,32.166348,10.0,12.430848,999.0,998.682344,9.0,12.947316,999.1,998.456759
0,427,100000,1,0.001,adam,max,1000,tanh,32,2000000,...,989.145855,29.443472,10.0,12.314452,999.0,998.694670,9.0,12.182960,999.1,998.535459
5,428,100000,1,0.001,adam,max,1000,tanh,32,2000000,...,988.958816,30.127724,10.0,12.331434,999.0,998.643546,9.0,11.566075,999.1,998.843393
4,429,100000,1,0.001,adam,max,1000,tanh,32,2000000,...,988.754586,29.172676,10.0,12.084761,999.0,998.791524,9.0,12.883305,999.1,998.470065
3,430,100000,1,0.001,adam,max,1000,tanh,32,2000000,...,989.016478,30.612021,10.0,12.307501,999.0,998.720030,9.0,12.484498,999.1,998.259424
1,431,100000,1,0.001,adam,max,1000,tanh,32,2000000,...,984.302091,35.068113,10.0,12.655791,999.0,998.683798,9.0,12.526569,999.1,998.241270
6,432,100000,1,0.001,adam,max,1000,tanh,32,2000000,...,989.281103,29.887601,10.0,12.368368,999.0,998.565276,9.0,12.280415,999.1,998.524679


## Merge current results with Final.csv and sort the result.

In [10]:
Final = pd.read_csv('Final.csv')

In [11]:
print('new data shape:{},old data shape:{}'.format(x.shape,Final.shape))
Final = pd.concat([Final,x])
print('new Full shape:',Final.shape)
Final.sort_values('experiment',inplace=True)
Final.to_csv('Final.csv',index=False)

new data shape:(7, 42),old data shape:(422, 42)
new Full shape: (429, 42)


In [12]:
pd.options.display.max_columns=80

In [13]:
Final[Final.experiment==380]

,experiment,reply_size,layers,tau,optimizer,advantage,max_steps,activation,batch_size,totalsteps,exploration,vanish,gamma,hidden_size,train_repeat,batch_norm,seed,rwrdschem,svision,details,tr_count,tr_TR,tr_TT,tr_TS,ts_count,ts_TR,ts_TT,ts_TS,tr_s,ts_s,tr_AR,tr_AS,ts_AR,ts_AS,tr_SMD,tr_SM,tr_RMD,tr_RM,ts_SMD,ts_SM,ts_RMD,ts_RM
374,380,100000,1,0.001,adam,max,1000,tanh,32,500000,1.0,0.75,0.99,100,1,False,6654,"[-10.0, 1000.0, -0.1]",180,punishment based on E355 CR:2,58892,8123699,5012,500007,5890,-493261,2170,62612,0.162569,0.211375,137.942318,8.490236,-83.745501,10.630221,7.0,7.607693,-0.6,256.046485,7.0,7.411192,-0.6,254.611679
